In [2]:
import pandas as pd
import nltk
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import re
import os.path

pd.options.display.max_colwidth = 150
pd.options.display.max_columns = 100

tokenizer = nltk.data.load('nltk:tokenizers/punkt/english.pickle')
tagdict = nltk.data.load('help/tagsets/upenn_tagset.pickle')
tagkeys = tagdict.keys()

In [3]:
def preprocess_data(filename):
    """
    This function takes in a filename and opens the file in read mode. Using the format provided in the sample CSV
    file, it removes extraneous characters and generates a list of lists where each list is a single row of the 
    input file.
    
    This list of lists is then converted into a Pandas DataFrame with the tweet id as the index.
    
    Input: fn (string)
    Output: tweets (Pandas DataFrame)
    """
    lines = []
    try:
        with open(filename, 'r') as fd:
            for line in fd:
                split_line = line.split(';') #columns are separated by semi-colons

                #remove extraneous characters from the line
                split_line2 = [item.replace('"', "") for item in split_line] 
                split_line2 = [item.replace('\r', "") for item in split_line2]
                split_line2 = [item.replace('\n', "") for item in split_line2]
                split_line2 = [re.sub(pattern=',+$', repl='', string=item) for item in split_line2] #remove all commas at end of line

                #tweets containing a comma are split into multiple strings in the list - here we concatenate them into one string
                if len(split_line2) > 3:
                    for item in range(3, len(split_line2)):
                        split_line2[2] = split_line2[2] + split_line2[item]
                    split_line2 = split_line2[0:3]
                lines.append(split_line2) #append each line to list

        #Convert list into dataframe
        tweets = pd.DataFrame(lines, columns=['id', 'polarity', 'tweet'])
        tweets.drop(labels=0, axis=0, inplace=True) #first list row is the header, dropping it

        tweets['id'] = tweets['id'].astype(int)
        tweets.set_index(keys='id', inplace=True) #the tweet id is being made the index
        tweets['polarity'] = tweets['polarity'].astype(int)
        
        return tweets
    except IOError, e:
        print 'File not found'

In [4]:
def generate_features(DF):
    """
    Generates features based on the tweets DataFrame. Calculates how many mentions, hashtags, and words in a tweet. 
    Also counts the different parts of speech each tweet contains.
    
    Input: tweets DataFrame
    Output: features DataFrame
    """
    #basic features such as the number of '@'-mentions, hashtags, and number of words in the tweet
    DF['mentions'] = DF['tweet'].apply(lambda s: len(re.findall(pattern= '@[a-z]+', string=s)))
    DF['hashtags'] = DF['tweet'].apply(lambda s: len(re.findall(pattern= '#[a-z]+', string=s)))
    tokenized = tweets['tweet'].apply(nltk.word_tokenize) #tokenise tweets into list of words
    DF['wordcount'] = tokenized.apply(len) #number of words in tweet
    
    #categorise tokens into relevant part of speech and store in DataFrame
    pos_list = tokenized.apply(lambda t: map(lambda x: x[1], nltk.pos_tag(t)))
    pos_dict = pos_list.apply(lambda x: Counter(x)) #Count how many of each type there are
    
    #convert the list of dicts into a dataframe
    pos_df = pd.DataFrame(columns=[], index=pos_dict.index.values, data=0)
    for i in pos_dict.index.values:
        for j in pos_dict[i].keys():
            pos_df.loc[i, j] = pos_dict[i][j]
    pos_df.replace(to_replace=np.nan, value=0, inplace=True) #replace any NaNs with 0 (meaning that part of speech wasn't in the tweet)
    pos_df = pos_df.astype(int) #convert everything to integer

    #merge the DataFrame with basic features and the parts of speech counts
    features = pd.merge(DF, pos_df, left_index=True, right_index=True)
    features.drop(labels=['tweet'], axis=1, inplace=True) #drop the original tweet text
    
    return features

In [11]:
fn = '/Users/navaneethan/Dropbox/projects/freshdesk/data/sts_gold_tweet.csv'

In [13]:
tweets = preprocess_data(filename=fn)
features = generate_features(DF=tweets)

In [15]:
try:
    input_file_path = os.path.dirname(os.path.abspath(fn))
    output_filename = input_file_path + '/' + 'tweet_features.csv'
    features.to_csv(output_filename)
except IOError, e:
    print 'Unable to write'

# Scrap

In [216]:
pos_dict[1]['DT']

3

In [2]:
fname = "/Users/navaneethan/Dropbox/projects/freshdesk/data/sts_gold_tweet2.txt"
tweets = pd.read_table(fname, delimiter=';', names=['id', 'polarity', 'tweet'])

In [ ]:
# pos_df = pd.DataFrame(columns=tagkeys, index=pos_dict.index.values, data=0)

In [461]:
vec = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english')
tf_idf_tweets = vec.fit_transform(tweets['tweet'])

UnicodeDecodeError: 'utf8' codec can't decode byte 0x89 in position 52: invalid start byte

In [227]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or